In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import json
import pandas as pd
sys.path.insert(0, '/Users/handanbinaerin/Library/Mobile Documents/com~apple~CloudDocs/music_edu')
from treat_song import GetSongInfo

file_path2 = 'processed_performance_data 2.json'
song_info_path = 'learning_points.json'

with open(file_path2, 'r') as file:
    data = json.load(file)

with open(song_info_path, 'r') as file:
    song_info = json.load(file)

In [13]:
sample = data[13] 

# 객체 생성
song_analyzer = GetSongInfo(sample, song_info)

# __call__ 메서드 사용 (객체를 함수처럼 호출)
result = song_analyzer(top_n=3)
result
# result['overall_summary']['pitch_accuracy_percent']

{'song_info': {'song_id': 'minuet', 'song_type': 'recorder'},
 'overall_summary': {'pitch_accuracy_percent': 0.0,
  'frequent_wrong_notes': [{'note': '파'}, {'note': '라'}, {'note': '솔'}]},
 'critical_measures': {'44': {'pitch_level': '중간 음역대',
   'pitch_variability': '음정 변화가 많음',
   'pitch_direction': '상행',
   'note_density': '빠른 패시지',
   'pitch_error': '매우 높음 (+13.1)',
   'frequent_wrong_syllables': ['도', '레', '미']},
  '0': {'pitch_level': '고음역대',
   'pitch_variability': '음정 변화가 많음',
   'pitch_direction': '복합적 이동',
   'note_density': '보통',
   'pitch_error': '매우 높음 (+12.1)',
   'frequent_wrong_syllables': ['높은 도', '파', '솔']},
  '42': {'pitch_level': '고음역대',
   'pitch_variability': '음정 변화 적음',
   'pitch_direction': '하행',
   'note_density': '보통',
   'pitch_error': '매우 높음 (+13.0)',
   'frequent_wrong_syllables': ['높은 도', '라#', '라']},
  '36': {'pitch_level': '고음역대',
   'pitch_variability': '음정 변화가 많음',
   'pitch_direction': '상행',
   'note_density': '빠른 패시지',
   'pitch_error': '매우 높음 (+12.3)

In [ ]:
for d in data:
  if d['song_id'] == 'overtherainbow':
    print(d['answers'][:5])

In [ ]:

# 결과 저장용 딕셔너리
all_results = {}

# 각 recording에 대해 분석 수행
for rec in recordings:
    answers = rec["answers"]
    measure_indices = rec["measure_indices"]
    performance_data = rec["performance_data"]
    recording_id = rec["recording_id"]

    result = prepare_llm_input(
        answers,
        measure_indices,
        performance_data,
        lyrics_per_measure=lyrics_per_measure,
        learning_points=learning_points
    )
    all_results[recording_id] = result

# recording_id=107에 대한 결과 예시 출력
import pprint
pprint.pprint(all_results[103], sort_dicts=False)


{'overall_summary': {'pitch_accuracy_percent': 34.23,
                     'frequent_wrong_notes': [{'note': '미'},
                                              {'note': '도'},
                                              {'note': '솔'}]},
 'critical_measures': {'16': {'pitch_level': '중간 음역대',
                              'pitch_variability': '음정 변화 적음',
                              'pitch_direction': '복합적 이동',
                              'note_density': '빠른 패시지',
                              'pitch_error': '매우 높음 (+1.9)',
                              'frequent_wrong_syllables': ['내(미)',
                                                           '사(솔)',
                                                           '-(미)'],
                              'lyrics': ['내', '사', '-', '랑', '-', '-', '아'],
                              'learning_point': '7~8마디처럼 숨 쉬는 위치가 다른 구간이므로, 숨 '
                                                '쉬는 타이밍을 정확히 파악하고 음악 흐름이 끊기지 '
                             

In [ ]:
song_type_dict = {'overtherainbow': 'western_vocal',
'dongmusaenggak': 'western_vocal',
'caromioben': 'western_vocal',
'grandpaclock': 'recorder',
'minuet': 'recorder',
'bibidi': 'recorder',
'monggeumpo': 'korean_vocal',
'jindo': 'korean_vocal',
'hangangsu': 'korean_vocal',
'taepyeongga': 'danso', 
'doraji': 'danso', 
'saeyasaeya': 'danso', }

# make the song_type to the csv file
import pandas as pd

# song_type_dict를 DataFrame으로 변환
df = pd.DataFrame(list(song_type_dict.items()), columns=['song_id', 'song_type'])

# DataFrame 확인
print("생성된 DataFrame:")
print(df)
print("\nDataFrame 정보:")
print(df.info())

# CSV 파일로 저장
csv_filename = 'song_instrument_mapping.csv'
df.to_csv(csv_filename, index=False, encoding='utf-8')
print(f"\nCSV 파일이 '{csv_filename}'로 저장되었습니다.")

생성된 DataFrame:
           song_id      song_type
0   overtherainbow  western_vocal
1   dongmusaenggak  western_vocal
2       caromioben  western_vocal
3     grandpaclock       recorder
4           minuet       recorder
5           bibidi       recorder
6       monggeumpo   korean_vocal
7            jindo   korean_vocal
8        hangangsu   korean_vocal
9      taepyeongga          danso
10          doraji          danso
11      saeyasaeya          danso

DataFrame 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   song_id    12 non-null     object
 1   song_type  12 non-null     object
dtypes: object(2)
memory usage: 320.0+ bytes
None

CSV 파일이 'song_instrument_mapping.csv'로 저장되었습니다.
